In [1]:
import pandas as pd

/tmp/ipykernel_2821826/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
MODEL_COLUMN = "model"
BASELINE="BEVfusion"
MODELS = ["CMT", "DeepInteraction", "TransFusion", "Sparsefusion", "BEVfusion"]
CORRUPTIONS=["beamsreducing", "brightness", "dark", "fog", "missingcamera", "motionblur", "pointsreducing", "snow", "spatialmisalignment", "temporalmisalignment"]
NICE_NAMES = {"beamsreducing": "Beams Red.",
              "brightness": "Brightness",
              "dark": "Darkness",
              "fog":"Fog",
              "missingcamera":
              "Missing Cam.",
              "motionblur": "Motion Blur",
              "pointsreducing": "Points Red.",
              "snow": "Snow",
              "spatialmisalignment": "Spatial Mis.",
              "temporalmisalignment": "Temporal Mis."
              }
SEVERITY_LEVELS = ["1", "2", "3"]

In [3]:
df = pd.read_csv("../evaluation/evaluations_nds.csv")

In [4]:
df = df.dropna()

In [5]:
df

,model,clean,temporalmisalignment_s1,temporalmisalignment_s2,temporalmisalignment_s3,spatialmisalignment_s1,spatialmisalignment_s2,spatialmisalignment_s3,pointsreducing_s1,pointsreducing_s2,...,dark_s3,brightness_s1,brightness_s2,brightness_s3,beamsreducing_s1,beamsreducing_s2,beamsreducing_s3,snow_s1,snow_s2,snow_s3
0,CMT,0.7290,0.6572,0.5680,0.4988,0.6462,0.5860,0.5367,0.7012,0.6834,...,0.6786,0.7284,0.7212,0.7037,0.6826,0.5873,0.4497,0.6483,0.6190,0.5554
1,DeepInteraction,0.6909,0.6139,0.5237,0.4544,0.5863,0.5014,0.4267,0.6522,0.6242,...,0.6254,0.6845,0.6767,0.6785,0.6162,0.4571,0.2839,0.5526,0.5311,0.4896
2,TransFusion,0.7084,0.6306,0.5437,0.4766,0.5708,0.4761,0.4095,0.6554,0.6190,...,0.6978,0.7080,0.7069,0.7056,0.6222,0.4591,0.2636,0.5447,0.5353,0.5087
3,Sparsefusion,0.7315,0.6523,0.5622,0.4901,0.6061,0.5149,0.4461,0.6881,0.6578,...,0.6978,0.7302,0.7257,0.7205,0.6593,0.5139,0.3380,0.5859,0.5704,0.5337
4,BEVfusion,0.7144,0.6341,0.5471,0.4085,0.5903,0.4943,0.4266,0.6684,0.6379,...,0.6861,0.7126,0.7053,0.6969,0.6396,0.4899,0.3199,0.5782,0.5572,0.5233


## Resistance Ability (RA)

In [6]:
for c in CORRUPTIONS:
    df[f"RA_{c}_s1"] = df[c+"_s1"] / df['clean']
    df[f"RA_{c}_s2"] = df[c+"_s2"] / df['clean']
    df[f"RA_{c}_s3"] = df[c+"_s3"] / df['clean']
    df[f"RA_{c}"] = (df[f"RA_{c}_s1"] + df[f"RA_{c}_s2"] + df[f"RA_{c}_s3"]) / 3
    
df["mRA"] = df[[f"RA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [7]:
df[[MODEL_COLUMN]+[f"RA_{c}_s1" for c in CORRUPTIONS] + [f"RA_{c}_s2" for c in CORRUPTIONS] + [f"RA_{c}_s3" for c in CORRUPTIONS] + ["mRA"]]

,model,RA_beamsreducing_s1,RA_brightness_s1,RA_dark_s1,RA_fog_s1,RA_missingcamera_s1,RA_motionblur_s1,RA_pointsreducing_s1,RA_snow_s1,RA_spatialmisalignment_s1,...,RA_brightness_s3,RA_dark_s3,RA_fog_s3,RA_missingcamera_s3,RA_motionblur_s3,RA_pointsreducing_s3,RA_snow_s3,RA_spatialmisalignment_s3,RA_temporalmisalignment_s3,mRA
0,CMT,0.936351,0.999177,0.967215,0.956653,0.984911,0.944582,0.961866,0.889300,0.886420,...,0.965295,0.930864,0.567078,0.961454,0.734842,0.875171,0.761866,0.736214,0.684225,0.869602
1,DeepInteraction,0.891880,0.990737,0.954407,0.837748,0.905051,0.948039,0.943986,0.799826,0.848603,...,0.982052,0.905196,0.210161,0.779129,0.714865,0.798668,0.708641,0.617600,0.657693,0.796483
2,TransFusion,0.878317,0.999435,0.991671,0.918408,0.990542,0.949605,0.925184,0.768916,0.805759,...,0.996047,0.985037,0.498165,0.980519,0.697911,0.752541,0.718097,0.578063,0.672784,0.824445
3,Sparsefusion,0.901299,0.998223,0.973616,0.924402,0.972932,0.956528,0.940670,0.800957,0.828571,...,0.984962,0.953930,0.520984,0.934518,0.734655,0.798086,0.729597,0.609843,0.669993,0.835279
4,BEVfusion,0.895297,0.997480,0.978443,0.917553,0.985162,0.960386,0.935610,0.809351,0.826288,...,0.975504,0.960386,0.501680,0.962066,0.766377,0.787794,0.732503,0.597144,0.571809,0.831630


In [17]:
for c in CORRUPTIONS:
    df[[MODEL_COLUMN]+[f"RA_{c}_s1", f"RA_{c}_s2", f"RA_{c}_s3"]].T.reset_index(drop=True).to_csv(f"../evaluation/ra_results_{c}.csv", index=False, header=False)

In [18]:
df[[MODEL_COLUMN]+[f"RA_{c}_s1" for c in CORRUPTIONS] + [f"RA_{c}_s2" for c in CORRUPTIONS] + [f"RA_{c}_s3" for c in CORRUPTIONS] + ["mRA"]].to_csv("../evaluation/ra_results.csv", index=False)

# Relative Resistance Ability (RRA)

In [10]:
for c in CORRUPTIONS:
    non_baseline = df.loc[df[MODEL_COLUMN] != BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1)
    baseline = df.loc[df[MODEL_COLUMN] == BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1).iloc[0]
    df[f"RRA_{c}"] = (((non_baseline / baseline) - 1)) * 100
    df.loc[df[MODEL_COLUMN] == BASELINE, f"RRA_{c}"] = 0
    
df["mRRA"] = df[[f"RRA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [11]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]]

,model,clean,RRA_beamsreducing,RRA_brightness,RRA_dark,RRA_fog,RRA_missingcamera,RRA_motionblur,RRA_pointsreducing,RRA_snow,RRA_spatialmisalignment,RRA_temporalmisalignment,mRRA
0,CMT,0.7290,18.642197,1.820503,-0.096334,9.398263,2.041012,-0.840834,8.212509,9.887261,17.052673,8.448135,7.456538
1,DeepInteraction,0.6909,-6.361253,-3.551163,-7.215452,-25.037221,-16.385588,-7.077023,-2.188219,-5.148610,0.211752,0.144681,-7.260810
2,TransFusion,0.7084,-7.209880,0.269529,1.146380,-0.552109,0.340169,-5.411524,-3.295704,-4.220172,-3.626257,3.849783,-1.870979
3,Sparsefusion,0.7315,4.263833,2.912805,1.820722,4.429280,0.297049,0.280278,3.242202,1.887020,3.699047,7.227779,3.006002
4,BEVfusion,0.7144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS]].T.to_csv("../evaluation/rra_nds_results.csv", header=False)